# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

TODO describe the liquidation logic in detail.

In [1]:
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

In [2]:
# Generate markets and prices
scenario = Scenario("baseline")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()  # Set block timestamps
scenario.prepare_for_trades(sample)  # Set External Market Price

# Unpack
markets = scenario.markets
liquidator = scenario.liquidator
controller = scenario.controller

[INFO][03:21:28][src.sim.scenario]-962383: Using 711360 1Inch quotes.
[INFO][03:21:28][src.sim.scenario]-962383: Fetching sim_market from subgraph.
[INFO][03:21:31][src.utils.poolgraph]-962383: Found 20 valid cycles of length 3.


In [3]:
# Artificially inflate collateral price
collateral = scenario.controller.COLLATERAL_TOKEN.address
sample = scenario.pricepaths[0]
prices_usd = sample.prices_usd
assert collateral in prices_usd.keys()
prices_usd[collateral] *= 2
sample.update(prices_usd)
scenario.update_market_prices(sample)

In [4]:
profit, count = liquidator.perform_liquidations(controller)
print(f"Liquidated {count} positions for a profit of {profit} USD.")

[INFO][03:21:31][src.agents.liquidator]-962383: There are 24 users to liquidate.
[INFO][03:21:31][src.agents.liquidator]-962383: Liquidating user 0xec718904654b29f1f571290259c324fd2e3aa63e with expected profit: 89054.440852.
[INFO][03:21:31][src.trades.cycle]-962383: Executing cycle Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=crvUSD, amt=77904451713), Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7f11cd1e85f0>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7f11b348add0>, amt=77961893995141982544573, frac=1000000000000000000, i=0, j=1), Swap(pool=External Market (USDC, WETH), in=Wrapped Ether, out=USD Coin, amt=37460935130686713385)], Expected Profit: 89054.440852).
[INFO][03:21:31][src.trades.cycle]-962383: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=crvUSD, amt=77904451713).
[INFO][03:21:31][src.trades.cycle]-962383: Executing trade Liquidation(con

[INFO][03:21:31][src.agents.liquidator]-962383: Liquidating user 0x167fc157aee30332f8a23b183c7c42a9c1d0854d with expected profit: 13315.609247.
[INFO][03:21:31][src.trades.cycle]-962383: Executing cycle Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=crvUSD, amt=12372609382), Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7f11cd1e85f0>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7f11b348a9d0>, amt=12378706682519017023967, frac=1000000000000000000, i=0, j=1), Swap(pool=External Market (USDC, WETH), in=Wrapped Ether, out=USD Coin, amt=5763722296757608380)], Expected Profit: 13315.609247).
[INFO][03:21:31][src.trades.cycle]-962383: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=crvUSD, amt=12372609382).
[INFO][03:21:31][src.trades.cycle]-962383: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7f1

Liquidated 24 positions for a profit of 2236673.233154 USD.


In [5]:
controller.users_to_liquidate()

[]

### A Quant Might Ask

Why are so many users eligible for liquidation? Wouldn't they already have been liquidated?

Yes. This is a known issue that is rooted in the crvusd subgraph. User positions are loaded in via two pathways:
1. We get `userState` snapshots from the subgraph, which snapshots a user's `depositedCollateral`, and `debt`. We use these values to load in their positions. Particularly the debt.
2. We get `bands` snapshots from the subgraph (a separate process at a different time). We use the relative `depositedCollateral` of each user to distribute the `stablecoin` and `collateral` from the `bandsSnapshot` to each user.

In theory, step (2) would correctly load in each user's share of `x,y` in LLAMMA so that we may correctly calculate their health and simulate their position. **HOWEVER** this is not the case, because of a bug in the subgraph. The bug is described in detail in this issue: https://github.com/curvefi/volume-subgraphs/issues/47.

At a high-level, bands with negative indices are being snapshot incorrectly due to the way negative integers are handled by the `graph-protocol` hexadecimal string class. When converting negative ints to hexadecimal strings, we are accidentally converting them into the positive representation. This means that if band `1` has 0 collateral and 0 stablecoin, and band `-1` has 1M collateral and 0 stablecoin, the snapshot will incorrectly tell us that band `-1` has 0, 0 instead of 1M, 0. 

This is what makes so many users *seem* like they are underwater.